### Perceptron - XOR

In [7]:
import torch

device = torch.device('cpu')
x = torch.FloatTensor([[0,0],[0,1],[1,0],[1,1]]).to(device)
y = torch.FloatTensor([[0],[1],[1],[0]]).to(device)

# nn Layers
linear = torch.nn.Linear(2,1, bias =True)
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(linear,sigmoid).to(device)

# cost, loss , optimizer 
criterion = torch.nn.BCELoss().to(device)  # 0,1  분류모델이므로 binary cross_entropy loss 사용
optimizer= torch.optim.SGD(model.parameters(), lr=1)

# train
for step in range(1000):
    optimizer.zero_grad()
    hypothesis = model(x)
    
    # cost, loss function
    cost = criterion(hypothesis, y)
    cost.backward()
    optimizer.step()
    if step % 100 == 0:
        print(step, cost.item())

0 0.7143567800521851
100 0.6931473016738892
200 0.6931471824645996
300 0.6931471824645996
400 0.6931471824645996
500 0.6931471824645996
600 0.6931471824645996
700 0.6931471824645996
800 0.6931471824645996
900 0.6931471824645996


In [9]:
print(hypothesis)
print(y)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
tensor([[0.],
        [1.],
        [1.],
        [0.]])


### Muti Layer Perceptron

- 여러개의 층을 가지는 perceptron
- 과거에는 Muti Layer Perceptron 를 학습할 수 있는 방법이 없었으나, Backpropagation 알고리즘이 개발되면서, Muti Layer model 에 대해서도 학습할 수 있게 되었다.  

- Backpropagation 이란, 어떤 입력 X 가 들어왔을 때 neural network을 통해서 output y를 구하게 되는데, 이 output과 원래 정답인 G(t)간의 차이인 cost(loss)에 대해서 neural network에 있는 weight 들에 대한 미분값을 계산하는 것이다. gradient를 가지고 뒷단에 있는 weight부터 loss를 최소화할 수 있도록 weight를 업데이트하는 방식


In [21]:
# 역전파를 직접 구현하는 코드

device = torch.device('cpu')
x = torch.FloatTensor([[0,0],[0,1],[1,0],[1,1]]).to(device)
y = torch.FloatTensor([[0],[1],[1],[0]]).to(device)

# nn Layers
# torch.nn.Linear를 안쓰고, weight, bias를 직접선언 해준다.
# 직접하는 이유는, 역전파를 직접 확인해 보기위해서
w1 = torch.Tensor(2,2).to(device)
b1 = torch.Tensor(2).to(device)
w2 = torch.Tensor(2,1).to(device)
b2 = torch.Tensor(1).to(device)


def sigmoid(x):
    return 1.0 / (1.0 + torch.exp(-x))
#     teturn torch.div(torch.tensor(1), torch.add(torch.tensor(1.0), torch.exp(-x)))

# sigmoid 를 미분했을 때 아래와 같은 형태를 가짐
def sigmoid_prime(x):
    return sigmoid(x) * (1- sigmoid(x))

# 학습
for step in range(10001):
    # forward
    l1 = torch.add(torch.matmul(x, w1),b1)
    a1 = sigmoid(l1)
    l2 = torch.add(torch.matmul(a1, w2), b2)
    y_pred = sigmoid(l2)
    
    # 아래의 cost 는 binary_cross_entropy loss 
    cost= -torch.mean(y * torch.log(y_pred) + (1-y) * torch.log(1-y_pred))
    
    
    # backprop (chain rule)  -> nn.backward로 한줄로 해결가능해짐! 
    
    #loss derivative
    
    # 아래의 식은 binary_cross entropy 를 미분한 식 
    # 1e-7은 0으로 나누어지는 것을 막기위한 것 
    d_y_pred = (y_pred - y) / (y_pred* (1.0- y_pred) + 1e-7)
    
    # layer 2
    d_l2 = d_y_pred * sigmoid_prime(l2)
    d_b2 = d_l2
    d_w2 = torch.matmul(torch.transpose(a1, 0, 1), d_b2)
    
    # layer1
    # 역전파를 위와 같은 방식으로 구해줌
    # transpose : 축을 지정해서, 두개의 값을 swap시켜라 라는 의미
    # matmul : 행렬 곱 
    d_a1 = torch.matmul(d_b2, torch.transpose(w2, 0, 1))
    d_l1 = d_a1 * sigmoid_prime(l1)
    d_b1 = d_l1
    d_w1 = torch.matmul(torch.transpose(x, 0, 1), d_b1)
    
    learning_rate = 1e-1
    
    # ------------------------------------------------------
    # weight, bias update 하는 코드  -> step 으로 한방에 해결 가능 
    
    # 기존의 weight에 역전파 weight * lr 을 곱해서 빼준다.
    # 아래는 gradient descent minimize를 시키는 경우고
    # 만약에 gradient ascent를 시키려면 + 로 바꿔준다.
    # 하지만, 보통은 minimize시키는 task이기 때문에 일반적으로 -라고 생각하면 된다.
    w1 = w1 - learning_rate * d_w1
    b1 = b1 - learning_rate * torch.mean(d_b1, 0)
    w2 = w2 - learning_rate * d_w2
    b2 = b2 - learning_rate * torch.mean(d_b2, 0)
    
    if step % 1000 == 0:
        print(step, cost.item())

In [30]:
# 기존의 torch.nn 으로 역전파 구하는 코드

device = torch.device('cpu')
x = torch.FloatTensor([[0,0],[0,1],[1,0],[1,1]]).to(device)
y = torch.FloatTensor([[0],[1],[1],[0]]).to(device)

# nn Layers
linear1 = torch.nn.Linear(2,2, bias=True)
linear2 = torch.nn.Linear(2,1, bias=True)
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid).to(device)

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

for step in range(10001):
    optimizer.zero_grad()  #  gradients를 clear해서 새로운 최적화 값을 찾기 위해 준비
    hypothesis = model(x)   # 준비한 데이터를 model 에 input 으로 넣어 output을 얻음
                            # # 이후, model에 넣어줌 keras는 보통 perdict , torch는 output이라 보통 씀
    cost = criterion(hypothesis, y)  # Model 에서 예측한 결과를 Loss Functoon 에 넣음
    cost.backward()             # Back Propagation을 통해 Gradients를 계산
    optimizer.step()            # 계산된 Gradients는 계산한 것으로 마무리 되는 것이 아니라 , Parameter 에 Update
    if step % 1000 == 0:
        print(step, cost.item())

0 0.7253798842430115
1000 0.019564880058169365
2000 0.006432583089917898
3000 0.003814517520368099
4000 0.0027047311887145042
5000 0.0020930441096425056
6000 0.0017060471000149846
7000 0.0014393834862858057
8000 0.0012445244938135147
9000 0.0010959478095173836
10000 0.0009789550676941872


In [31]:
print(hypothesis)
print(y)

tensor([[0.0011],
        [0.9991],
        [0.9991],
        [0.0010]], grad_fn=<SigmoidBackward>)
tensor([[0.],
        [1.],
        [1.],
        [0.]])
